<a href="https://colab.research.google.com/github/aaririri/task8_KrackhardtKite/blob/main/task8_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install torch torchvision torchaudio torch-geometric networkx

import torch
from torch_geometric.datasets import TUDataset
import networkx as nx
import random
from networkx.algorithms.graph_hashing import weisfeiler_lehman_graph_hash as WLHash

In [10]:
dataset = TUDataset(root="/content/data", name="AIDS")
print("Dataset loaded")
print("Total graphs in dataset:", len(dataset))

graphs_nx = []
for data in dataset:
    G = nx.Graph()
    edge_index = data.edge_index.numpy()
    for i in range(data.num_nodes):
        G.add_node(i, label=int(data.x[i][0]))
    for i, j in edge_index.T:
        G.add_edge(int(i), int(j))
    graphs_nx.append(G)

print("Converted dataset to NetworkX graphs")
print("Example graph nodes with labels:", list(graphs_nx[0].nodes(data=True))[:5])
print("Example graph edges:", list(graphs_nx[0].edges())[:5])

Dataset loaded
Total graphs in dataset: 2000
Converted dataset to NetworkX graphs
Example graph nodes with labels: [(0, {'label': 1}), (1, {'label': 1}), (2, {'label': 1}), (3, {'label': 1}), (4, {'label': 1})]
Example graph edges: [(0, 1), (0, 5), (1, 2), (1, 11), (2, 3)]


In [11]:
wl_hashes = [WLHash(graph, node_attr="label") for graph in graphs_nx]
print("Computed WL hashes for all graphs")

groups = {}
for idx, h in enumerate(wl_hashes):
    groups.setdefault(h, []).append(idx)

largest_group = max(groups.values(), key=len)
print("Largest WL-isomorphic group size:", len(largest_group))
print("Graph indices in largest group:", largest_group)

Computed WL hashes for all graphs
Largest WL-isomorphic group size: 10
Graph indices in largest group: [142, 263, 277, 373, 595, 683, 1317, 1374, 1659, 1909]


In [12]:

def perturb_remove_edge(G):
    G_copy = G.copy()
    if G_copy.number_of_edges() > 0:
        edge_to_remove = random.choice(list(G_copy.edges()))
        G_copy.remove_edge(*edge_to_remove)
    return G_copy

perturbed_graphs = []
for idx in largest_group:
    original_graph = graphs_nx[idx]
    for _ in range(10):
        perturbed_graphs.append(perturb_remove_edge(original_graph))

print("Generated perturbed graphs:", len(perturbed_graphs))

Generated perturbed graphs: 100


In [13]:
all_graphs = [graphs_nx[idx] for idx in largest_group] + perturbed_graphs
print("Combined originals and perturbed graphs")
print("Total graphs in analysis set:", len(all_graphs))



Combined originals and perturbed graphs
Total graphs in analysis set: 110


In [14]:
all_wl_hashes = [WLHash(g, node_attr="label") for g in all_graphs]
print("Recomputed WL hashes for originals and perturbed graphs")

original_hash = WLHash(graphs_nx[largest_group[0]], node_attr="label")
same_iso = sum(1 for h in all_wl_hashes if h == original_hash)
different_iso = len(all_wl_hashes) - same_iso

print("Final WL comparison results")
print("Total graphs tested:", len(all_graphs))
print("Number of graphs still isomorphic:", same_iso)
print("Number of broken isomorphisms:", different_iso)


Recomputed WL hashes for originals and perturbed graphs
Final WL comparison results
Total graphs tested: 110
Number of graphs still isomorphic: 10
Number of broken isomorphisms: 100


In [15]:
print("Checking first 10 perturbed graphs individually")
for i, g in enumerate(perturbed_graphs[:10]):
    h = WLHash(g, node_attr="label")
    status = "Still Isomorphic" if h == original_hash else "Broken"
    print("Perturbed Graph", i, "→", status)


Checking first 10 perturbed graphs individually
Perturbed Graph 0 → Broken
Perturbed Graph 1 → Broken
Perturbed Graph 2 → Broken
Perturbed Graph 3 → Broken
Perturbed Graph 4 → Broken
Perturbed Graph 5 → Broken
Perturbed Graph 6 → Broken
Perturbed Graph 7 → Broken
Perturbed Graph 8 → Broken
Perturbed Graph 9 → Broken
